<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#加载数据和模型" data-toc-modified-id="加载数据和模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>加载数据和模型</a></span></li><li><span><a href="#精度，召回率，F和时间随参数的变化" data-toc-modified-id="精度，召回率，F和时间随参数的变化-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>精度，召回率，F和时间随参数的变化</a></span><ul class="toc-item"><li><span><a href="#随MIN_LEN变化" data-toc-modified-id="随MIN_LEN变化-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>随MIN_LEN变化</a></span></li><li><span><a href="#随MIN_COVER_ANGLE变化" data-toc-modified-id="随MIN_COVER_ANGLE变化-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>随MIN_COVER_ANGLE变化</a></span></li><li><span><a href="#随MIN_COVER_RHO1变化" data-toc-modified-id="随MIN_COVER_RHO1变化-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>随MIN_COVER_RHO1变化</a></span></li><li><span><a href="#随MIN_COVER_RHO2变化" data-toc-modified-id="随MIN_COVER_RHO2变化-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>随MIN_COVER_RHO2变化</a></span></li></ul></li></ul></div>

In [2]:
%matplotlib inline
dataset = '02_prasad'
raw_path = f'/home/liucc/Workspace/Ellipse/03_results/{dataset}/01_raw/'
label_path = f'/home/liucc/Workspace/Ellipse/03_results/{dataset}/04_ellipse_label/'

In [3]:
import os 
fnames = os.listdir(raw_path)
fnames = [n[:-4] for n in fnames if n[-3:]=='jpg']
fnames.sort()
print(fnames[:10])

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010']


# 加载数据和模型

In [4]:
import sys
sys.path.append('/home/liucc/Workspace/Ellipse/01_data')
from cc_process_image import *
import imageio

raws = get_all_images(f'{raw_path}',fnames,'jpg')

print(len(raws))

/home/liucc/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/home/liucc/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


198


In [5]:
gts = []

for name in fnames:
    lname = label_path+name+'.txt'
    
    with open(lname,'r') as f:
        txt = f.read()
        
        ells_t = [[float(v) for v in l.split(' ')] for l in txt.split('\n')]
        
        gts.append(ells_t)
        

print(len(gts))

198


# 精度，召回率，F和时间随参数的变化

In [6]:
import sys
sys.path.append('/home/liucc/Workspace/Ellipse/02_algorithm/02_ellipse_detect')
from cc_ellipse import *
sys.path.append('../')
from experiment import *

In [15]:
import time
from skimage.feature import canny



def one_task(glob):
    start = time.time()

    prs = []
    
    for im in raws:
        im = preprocess_image(im)
        imo = canny(im, sigma=1.5)
        ells = detect_ellipses_im(imo,glob)
        
        prs.append(ells)

    end = time.time()
    
    costT = (end-start)/len(raws)
    
    precision,recall,F1,_,_ = cal_metrics(prs,gts,threshold=0.8)
    
    
    return [precision,recall,F1,costT]

In [20]:
#全局参数
GLOB = {
    'MIN_LEN' : 80,
    'MIN_COVER_ANGLE' : 100,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.2,
}
precision,recall,F1,costT = one_task(GLOB)

In [21]:
print('Precision:%.4f' % precision)
print('Recall:   %.4f' % recall)
print('F1 Score: %.4f' % F1)
print('Avg Time: %.0f' % (costT*1000))

Precision:0.6767
Recall:   0.3077
F1 Score: 0.4230
Avg Time: 129


## 随MIN_LEN变化

In [23]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MIN_LEN' : 0,
    'MIN_COVER_ANGLE' : 0,
    'MIN_COVER_RHO1' : 0,
    'MIN_COVER_RHO2' : 0,
}

res1 = []
xs1 = list(range(50,100,10))
for x in xs1:
    GLOB['MIN_LEN'] = x
    res = one_task(GLOB)
    res1.append(res)
    print(f'{x}:{res}')

/home/liucc/Workspace/Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse.py:515: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


50:[0.24121146565711196, 0.3811965811965812, 0.2954620735342829, 0.16528422302669948]
60:[0.282437745740498, 0.3683760683760684, 0.3197329376854599, 0.15273503944127245]
70:[0.33068992862807295, 0.3564102564102564, 0.3430686960098725, 0.14614953416766543]
80:[0.3699421965317919, 0.3282051282051282, 0.34782608695652173, 0.13312849372324317]
90:[0.3986095017381228, 0.294017094017094, 0.33841613379242497, 0.13612307081318864]


## 随MIN_COVER_ANGLE变化

In [17]:
GLOB = {
    'MIN_LEN' : 80,
    'MIN_COVER_ANGLE' : 0,
    'MIN_COVER_RHO1' : 0,
    'MIN_COVER_RHO2' : 0,
}

res2 = []
xs2 = np.arange(90,140,10)
for x in xs2:
    GLOB['MIN_COVER_ANGLE'] = x
    res = one_task(GLOB)
    res2.append(res)
    print(f'{x}:{res}')

90:[0.41881298992161253, 0.31965811965811963, 0.36257876878332523, 0.1270626157221168]
100:[0.4372801875732708, 0.3188034188034188, 0.36875926841324763, 0.12799056491466484]
110:[0.4475955610357583, 0.31025641025641026, 0.3664815749621404, 0.13311345649488043]
120:[0.4601542416452442, 0.305982905982906, 0.3675564681724846, 0.1289562950230608]
130:[0.4791386271870794, 0.30427350427350425, 0.3721902770517511, 0.13015471323572023]


## 随MIN_COVER_RHO1变化

In [18]:
GLOB = {
    'MIN_LEN' : 80,
    'MIN_COVER_ANGLE' : 100,
    'MIN_COVER_RHO1' : 0.0,
    'MIN_COVER_RHO2' : 0.0,
}

res3 = []
xs3 = np.arange(0.2,0.7,0.1)
for x in xs3:
    GLOB['MIN_COVER_RHO1'] = x
    res = one_task(GLOB)
    res3.append(res)
    print(f'{x}:{res}')

0.2:[0.5344827586206896, 0.31794871794871793, 0.39871382636655944, 0.12936915773333926]
0.30000000000000004:[0.5885167464114832, 0.3153846153846154, 0.41068447412353926, 0.129606215640752]
0.4000000000000001:[0.6460176991150443, 0.31196581196581197, 0.4207492795389049, 0.1315648363094137]
0.5000000000000001:[0.6794625719769674, 0.30256410256410254, 0.41868716735659367, 0.12892276349693838]
0.6000000000000001:[0.7134020618556701, 0.29572649572649573, 0.4181268882175227, 0.18711909501239507]


## 随MIN_COVER_RHO2变化

In [19]:
GLOB = {
    'MIN_LEN' : 80,
    'MIN_COVER_ANGLE' : 100,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.0,
}

res4 = []
xs4 = np.arange(0,0.6,0.1)
for x in xs4:
    GLOB['MIN_COVER_RHO2'] = x
    res = one_task(GLOB)
    res4.append(res)
    print(f'{x}:{res}')

0.0:[0.6460176991150443, 0.31196581196581197, 0.4207492795389049, 0.21689000274195816]
0.1:[0.6460176991150443, 0.31196581196581197, 0.4207492795389049, 0.1638105565851385]
0.2:[0.6766917293233082, 0.3076923076923077, 0.42303172737955347, 0.12973227284171365]
0.30000000000000004:[0.7315010570824524, 0.29572649572649573, 0.4211807668898357, 0.21653174751936788]
0.4:[0.821522309711286, 0.2675213675213675, 0.4036105738233398, 0.22014996138485995]
0.5:[0.8677966101694915, 0.2188034188034188, 0.34948805460750854, 0.21678870494919594]
